1. 不考虑词性，将分词形式有争议的词挑出
为了达到这个目的，首先将分词剔除

In [1]:
def remove_pos(line):
    if line[-1] == '\n':
        line = line[:-1]
    if line[-1] == ' ':
        line = line[:-1]
    word_list = line.split(" ")
    out_line = ""
    for i in range(len(word_list)):
        try:
            out_line += word_list[i][:word_list[i].index("/")] + " "
        except:
            print(word_list)
            print(i)
            print(word_list[i-20:i])
    out_line = out_line[:-1]
    return out_line
    
remove_pos("屹立/v 在/p 烟波/n 浩瀚/v 的/u 伶仃洋/n 上/f")

'屹立 在 烟波 浩瀚 的 伶仃洋 上'

In [ ]:
GLOBAL_ENCODING = "utf-8-sig"

In [ ]:
# 以下将分词不一样的两行挑出来，写入 seg_diff
for i in range(1,138):
    I_file = open("I_study\\study" + str(i) + ".txt", encoding=GLOBAL_ENCODING)
    S_file = open("S_study\\study" + str(i) + ".txt", encoding=GLOBAL_ENCODING)

    with open("seg_diff\\study" + str(i) + ".txt", 'w', encoding=GLOBAL_ENCODING) as diff:
        while True:
            I_line = I_file.readline()
            S_line = S_file.readline()
            if len(I_line) < 2 or len(S_line) < 2:
                break
#             try:
            I_no_pos = remove_pos(I_line)
            S_no_pos = remove_pos(S_line)
#             except:
#                 print(I_line)
#                 print("\n")
#                 print(S_line)
#                 print("\n")
#                 print("\n")

            if I_no_pos != S_no_pos:
                s = diff.write(I_no_pos)
                s = diff.write('\n')
                s = diff.write(S_no_pos)
                s = diff.write('\n')
                s = diff.write('\n')

    I_file.close()
    S_file.close()


### 以下将有分歧的词挑出来，写入word_diff

In [ ]:
for i in range(1,138):
    print("--------------------------------------\n"+str(i)+'\n----------------------------\n')
    f = open("seg_diff\\study" + str(i) + ".txt", encoding=GLOBAL_ENCODING)
    
    with open("word_diff\\study" + str(i) + ".txt", 'w', encoding=GLOBAL_ENCODING) as wd:
        while True:
            I_line = f.readline()
            if I_line == '':
                break
            S_line = f.readline()[:-1]
            I_line = revised_line(I_line[:-1], S_line).split(" ")
            S_line = S_line.split(" ")
            e = f.readline()

            i_idx = 0
            s_idx = 0
            i_str = ""
            s_str = ""
            w_i_str = ""
            w_s_str = ""
            while i_idx < len(I_line) and s_idx < len(S_line):
                try:
                    if I_line[i_idx] == S_line[s_idx]:
                        g = nd.write(I_line[i_idx] + " ")                     
                    if i_str == s_str:
                        if w_i_str != w_s_str:
                            g = wd.write("<words>" + i_str + '\n')
                            g = wd.write("<I_seg>" + w_i_str + '\n')
                            g = wd.write("<S_seg>" + w_s_str + '\n')
                            g = wd.write("\n")
                            i_str = ""
                            s_str = ""
                            w_i_str = ""
                            w_s_str = ""
                            i_idx += 1
                            s_idx += 1
                        elif len(I_line[i_idx]) == len(S_line[s_idx]):
                            i_idx += 1
                            s_idx += 1
                        else:
                            i_str += I_line[i_idx]
                            s_str += S_line[s_idx]
                            w_i_str += I_line[i_idx]
                            w_s_str += S_line[s_idx]
                    elif len(i_str) > len(s_str):
                        s_idx += 1
                        s_str += S_line[s_idx]
                        w_s_str += " " + S_line[s_idx]
                    else:
                        i_idx += 1
                        i_str += I_line[i_idx]
                        w_i_str += " " + I_line[i_idx]


                except:
#                     print(I_line)
#                     print(S_line)
#                     print(i_idx)
#                     print(s_idx)
#                     print(i_str + '\n')
#                     print(s_str + '\n')
#                     print(w_i_str + '\n')
#                     print(w_s_str + '\n')
    #                 sys.exit(0)
                    break
                

## 接下来针对有歧义的词进行全盘搜索

### 参考stats.py
### 注: stats.py 里有个mword.txt, 里面有一些统计结果，这个是曾经的一次错误统计（分词形式没错，但词性有错）因此第二次统计的时候直接用了这些词，这里直接当作是word_diff 中有分歧的那些词即可

## 运行完stats.py后会会得到许多小字典，将它们合成一个大字典,然后计算词频得到 “new_big_dict_ratio.txt”

## stats.py的运行时间约为 8小时

# -----------------------------------------------------------------------------------------

# 首先将统计出的数据做成一个超大的字典
* 这样以后可以直接读取json形式，或用python的eval()函数
* 下面big_dict_ratio在统计频数的基础之上加了每个词占总出现次数的比值，以便之后进一步分析

In [ ]:
big_dict = dict()

# with open("big_dict.txt", 'w', encoding="utf-8-sig") as bd:
for i in range(1,17):
    print(i)
    f_name = "big_dict_data\\" + str(i) + ".txt"
    with open(f_name, encoding="utf-8-sig") as fn:
        while True:
            l = fn.readline()
            if len(l) == 0:
                break
            l = l[:-1]
            pair = l.split("\t")
            big_dict[pair[0]] = eval(pair[1])
        # g = bd.write(str(big_dict))

for key in big_dict:
    times = big_dict[key]['a_time']
    for i_seg in big_dict[key]['i_segs']:
        i_frq = big_dict[key]['i_segs'][i_seg]['frq']
        i_ratio = round(i_frq/times, 3)
        big_dict[key]['i_segs'][i_seg]['rat'] = i_ratio
    for s_seg in big_dict[key]['s_segs']:
        s_frq = big_dict[key]['s_segs'][s_seg]['frq']
        s_ratio = round(s_frq/times, 3)
        big_dict[key]['s_segs'][s_seg]['rat'] = s_ratio

with open("new_big_dict_ratio.txt", 'w', encoding="utf-8-sig") as bdr:
    g = bdr.write(str(big_dict))

## 直接将数据读取成字典，方便以后使用

In [ ]:
big_dict = dict()
with open("new_big_dict_ratio.txt", encoding='utf-8-sig') as bdr:
    big_dict = eval(bdr.read())

In [ ]:
big_dict['王某']

# ictclas 中的个别字在分词时会进行简繁转换，在此将其剔除

In [ ]:
count = 0
for k,v in list(big_dict.items()):
    if len(v['i_segs']) == 0:
        print(k)
        del big_dict[k]
        count += 1
print(count)

# 争议很大的情况 -- 写入 "close.txt" 文档
* 对于某个词来说，如果两个分词器对其的所有分词形中，出现最多(比率之和大于0.9)的两种形式不相同
* 则判定这个词对两个分词系统存在争议

In [ ]:
count = 0
with open("close.txt", 'w', encoding="utf-8-sig") as cc:
    for k in big_dict:

        i_max_rat = max(big_dict[k]['i_segs'].values(), key=lambda x:x['rat'])['rat']
        s_max_rat = max(big_dict[k]['s_segs'].values(), key=lambda x:x['rat'])['rat']

        # 对于某个词来说，如果两个分词器对其的所有分词形中，出现最多(比率之和大于0.9)的两种形式不相同
        # 则判定这个词对两个分词系统存在争议big
        if i_max_rat + s_max_rat >= 0.9:
            count += 1
            word_dict = big_dict[k]
            i_word = max(word_dict['i_segs'].items(), key=lambda x:x[1]['rat'])[0]
            s_word = max(word_dict['s_segs'].items(), key=lambda x:x[1]['rat'])[0]
            if i_word == s_word:
                continue
                
            tmp_i_dic = dict()
            tmp_s_dic = dict()
            
            for ipos in word_dict['i_segs']:
                tmp_i_dic[ipos] = str(word_dict['i_segs'][ipos]['frq']) + " (" + str(word_dict['i_segs'][ipos]['rat']) + ")" 
            for spos in word_dict['s_segs']:
                tmp_s_dic[spos] = str(word_dict['s_segs'][spos]['frq']) + " (" + str(word_dict['s_segs'][spos]['rat']) + ")"
            
            g = cc.write(k + ": " + str(big_dict[k]["a_time"]) + '\n')
            g = cc.write("i_segs: " + str(tmp_i_dic) + '\n')
            g = cc.write("s_segs: " + str(tmp_s_dic) + '\n')
            g = cc.write("\n")
print("共" + str(count) + "个词") # 共13758个词

# 针对模棱两可的情况 -- 写入 "undecidable.txt" 文档

In [ ]:
count = 0
with open("undecidable.txt", 'w', encoding="utf-8-sig") as cc:
    for k in big_dict:
        i_max_rat = max(big_dict[k]['i_segs'].values(), key=lambda x:x['rat'])['rat']
        s_max_rat = max(big_dict[k]['s_segs'].values(), key=lambda x:x['rat'])['rat']
        if i_max_rat + s_max_rat < 0.65:
            
            word_dict = big_dict[k]
            i_word = max(word_dict['i_segs'].items(), key=lambda x:x[1]['rat'])[0]
            s_word = max(word_dict['s_segs'].items(), key=lambda x:x[1]['rat'])[0]
            if i_word == s_word:
                continue
            count += 1
            tmp_i_dic = dict()
            tmp_s_dic = dict()
            
            for ipos in word_dict['i_segs']:
                tmp_i_dic[ipos] = str(word_dict['i_segs'][ipos]['frq']) + " (" + str(word_dict['i_segs'][ipos]['rat']) + ")" 
            for spos in word_dict['s_segs']:
                tmp_s_dic[spos] = str(word_dict['s_segs'][spos]['frq']) + " (" + str(word_dict['s_segs'][spos]['rat']) + ")"
            
            g = cc.write(k + ": " + str(big_dict[k]["a_time"]) + '\n')
            g = cc.write("i_segs: " + str(tmp_i_dic) + '\n')
            g = cc.write("s_segs: " + str(tmp_s_dic) + '\n')
            g = cc.write("\n")
print("共" + str(count) + "个词") # 共120个词

### 曹教授关于模棱两可的处理的四条建议：

*** 建议之一

先看两个例子：

1. 将要有时是词，有时不是

将要: 475
i_segs: {'将 要': 178, '将要': 297}
s_segs: {'将要': 314, '将 要': 161}

2. 就是，同上

就是: 47966
i_segs: {'就 是': 34408, '就是': 13558}
s_segs: {'就是': 27860, '就 是': 20106}

因此，需要输出上述词的前后文，帮助判断。

*** 建议之二

代词+名词，则不合并。例如，

此事: 1433
i_segs: {'此事': 720, '此 事': 713}
s_segs: {'此 事': 909, '此事': 524}

*** 建议三
百家姓+某
老+百家姓
小+百家姓
则不合并。例如，

谢 某
周 某
都是正确的分词。

*** 建议四

副词+动词
副词+形容词
副词+介词
则不合并。例如，

不知情: 226
i_segs: {'不 知情': 76, '不知 情': 127, '不 知 情': 23}
s_segs: {'不知情': 134, '不 知情': 92}

不够: 3768
i_segs: {'不 够': 2056, '不够': 1712}
s_segs: {'不够': 2297, '不 够': 1471}

竟/d 在/p
是正确的分词。

# 针对四条处理建议:
* 建议一的情况需要人工逐个考察，因此要首先处理，处理完成后将这些词从模棱两可的集合中去除，避免重复处理

### 以下代码中，第一个for循环自动生成建议二的config
### 第二个for循环自动生成建议四的config

In [ ]:
def word_only(item):
    item_l = item.split(" ")
    item_str = ""
    for il in item_l:
        item_str += il[:il.index("/")] + " "
    return item_str[:-1]

In [ ]:
count = 0
with open("config_rule_undicibale.txt", 'w', encoding="utf-8-sig") as cu:
    for k in big_dict:
        i_max_rat = max(big_dict[k]['i_segs'].values(), key=lambda x:x['rat'])['rat']
        s_max_rat = max(big_dict[k]['s_segs'].values(), key=lambda x:x['rat'])['rat']
        if i_max_rat + s_max_rat < 0.65 and len(k) == 2:
            i_dic = big_dict[k]['i_segs']
            s_dic = big_dict[k]['s_segs']
            
            need_config = False
            for segs in i_dic:
                max_p = ""
                p_num = 0
                for pos in i_dic[segs]['pos']:
                    if i_dic[segs]['pos'][pos] > p_num:
                        max_p = pos
                    
                chars = max_p.split(" ")
                if len(chars) == 2 and '/r' in chars[0] and 'n' in chars[1]:
                    need_config = True
            for segs in s_dic:
                max_p = ""
                p_num = 0
                for pos in s_dic[segs]['pos']:
                    if s_dic[segs]['pos'][pos] > p_num:
                        max_p = pos
                    
                chars = max_p.split(" ")
                if len(chars) == 2 and '/r' in chars[0] and 'n' in chars[1]:
                    need_config = True
            if need_config:
                g = cu.write("//" + k + "\n")
#                 g = cu.write(str(big_dict[k]) + '\n')
                correct_exp = k[0] + '/r' + ' ' + k[1] + '/n'
                pos_set = set()
                for segs in i_dic:
                    for pos in i_dic[segs]['pos']:
                        pos_set.add(pos)
                for item in pos_set:
                    if word_only(item) != word_only(correct_exp):
                        g = cu.write('KEY' + item.replace(" ", "+") + "->[" + item + "]MERGE[" + correct_exp + "]\n")
                g = cu.write("\n")
    
    for k in big_dict:
        i_max_rat = max(big_dict[k]['i_segs'].values(), key=lambda x:x['rat'])['rat']
        s_max_rat = max(big_dict[k]['s_segs'].values(), key=lambda x:x['rat'])['rat']
        if i_max_rat + s_max_rat < 0.65 and len(k) == 2:
            i_dic = big_dict[k]['i_segs']
            s_dic = big_dict[k]['s_segs']
            
            need_config = False
            correct_exp = ""
            for segs in i_dic:
                max_p = ""
                p_num = 0
                for pos in i_dic[segs]['pos']:
                    if i_dic[segs]['pos'][pos] > p_num:
                        max_p = pos
                    
                chars = max_p.split(" ")
                if len(chars) == 2 and '/d' in chars[0]:
                    if '/v' in chars[1]:
                        need_config = True
                        correct_exp = k[0] + '/d' + ' ' + k[1] + '/v'
                    if '/a' in chars[1]:
                        need_config = True
                        correct_exp = k[0] + '/d' + ' ' + k[1] + '/a'
                    if 'p' in chars[1]:
                        need_config = True
                        correct_exp = k[0] + '/d' + ' ' + k[1] + '/p'
                    
            for segs in s_dic:
                max_p = ""
                p_num = 0
                for pos in s_dic[segs]['pos']:
                    if s_dic[segs]['pos'][pos] > p_num:
                        max_p = pos
                    
                chars = max_p.split(" ")
                if len(chars) == 2 and '/d' in chars[0]:
                    if '/v' in chars[1]:
                        need_config = True
                        correct_exp = k[0] + '/d' + ' ' + k[1] + '/v'
                    if '/a' in chars[1]:
                        need_config = True
                        correct_exp = k[0] + '/d' + ' ' + k[1] + '/a'
                    if 'p' in chars[1]:
                        need_config = True
                        correct_exp = k[0] + '/d' + ' ' + k[1] + '/p'
                    
            if need_config:
                g = cu.write("// " + k + "\n")
#                 g = cu.write(str(big_dict[k]) + '\n')
                pos_set = set()
                for segs in i_dic:
                    for pos in i_dic[segs]['pos']:
                        pos_set.add(pos)
                for item in pos_set:
                    g = cu.write('KEY' + item.replace(" ", "+") + "->[" + item + "]MERGE[" + correct_exp + "]\n")
                g = cu.write("\n")

# 生成config_confident 和 config_conflict

In [ ]:
def seg_w_any(seg):
    l = seg.split(" ")
    s = ""
    for w in l:
        s += w + "/ANY "
    return s[:-1]

In [ ]:
nd_count = 0
wd_count = 0
_encoding = "utf-8-sig"

nd = open("need_discuss_confident.txt",'w',encoding=_encoding)
wd = open("config_confident.txt", 'w', encoding=_encoding)

co = open("need_discuss_conflict.txt", 'w', encoding=_encoding)
co_count = 0
co_dict = dict()

cciu = open("config_conflict_I_u.txt", 'w', encoding=_encoding)
ccip = open("config_conflict_I_p.txt", 'w', encoding=_encoding)
ccif = open("config_conflict_I_f.txt", 'w', encoding=_encoding)
cciy = open("config_conflict_I_y.txt", 'w', encoding=_encoding)
ccim = open("config_conflict_I_m.txt", 'w', encoding=_encoding)
ccsu = open("config_conflict_S_u.txt", 'w', encoding=_encoding)
ccsp = open("config_conflict_S_p.txt", 'w', encoding=_encoding)
ccsf = open("config_conflict_S_f.txt", 'w', encoding=_encoding)
ccsy = open("config_conflict_S_y.txt", 'w', encoding=_encoding)
ccsm = open("config_conflict_S_m.txt", 'w', encoding=_encoding)

rn = open("config_r_n.txt", 'w', encoding=_encoding)

ad_v = open("config_adverb_v.txt", 'w', encoding=_encoding)
ad_a = open("config_adverb_a.txt", 'w', encoding=_encoding)
ad_p = open("config_adverb_p.txt", 'w', encoding=_encoding)

en = open("need_discuss_else.txt", 'w', encoding=_encoding)
en_dict = dict()

cciu_count = 0
ccip_count = 0
ccif_count = 0
cciy_count = 0
ccim_count = 0
ccsu_count = 0
ccsp_count = 0
ccsf_count = 0
ccsy_count = 0
ccsm_count = 0

rn_count = 0

ad_v_count = 0
ad_a_count = 0
ad_p_count = 0

en_count = 0

for item in big_dict:

    word_dict = big_dict[item]
    i_segs = big_dict[item]["i_segs"]
    s_segs = big_dict[item]["s_segs"]

    i_cons = dict()
    s_cons = dict()

    for i in i_segs:
        i_cons[i] = i_segs[i]['rat']
    for s in s_segs:
        s_cons[s] = s_segs[s]['rat']
        
#     print(i_cons)
#     print(s_cons)


    i_con = max(i_cons.values())
    s_con = max(s_cons.values())

    if abs(i_con - s_con) >= 0.175:
        correct_form = ""
        if i_con > s_con:
            correct_seg = max(i_cons, key=i_cons.get)
            poss = i_segs[correct_seg]['pos']
            check = sorted(poss.values())
            if check[-1] / sum(check) < 0.8:
                g = nd.write(item + ":" + str(poss) + "\n\n")
                nd_count += 1
            else:
                correct_form = max(poss, key=poss.get)
        else:
            correct_seg = max(s_cons, key=s_cons.get)
            poss = s_segs[correct_seg]['pos']
            check = sorted(poss.values())
            if check[-1] / sum(check) < 0.8:
                g = nd.write(item + ":" + str(poss) + "\n\n")
                nd_count += 1
            else:
                correct_form = max(poss, key=poss.get)

#         KEY来自于/ANY->[来自于/ANY]MERGE[来自/v 于/p]
        if correct_form != "":

            correct_form = ict_pku(correct_form)

            wd_count += 1
            g = wd.write("// " + item + '\n')

            seg_set = set()
            for segs in i_cons:
                seg_set.add(segs)
            for segs in s_cons:
                seg_set.add(segs)

            for segs in seg_set:
                if segs != correct_seg:
                    g = wd.write("KEY" + seg_w_any(segs).replace(" ", "+") + "->[" + seg_w_any(segs) + "]MERGE[" + correct_form + "]\n")
            g = wd.write("\n")

    # 争议很大
    elif i_con + s_con >= 0.95:
        i_rep_form = max(i_cons, key=i_cons.get)
        s_rep_form = max(s_cons, key=s_cons.get)
        if i_rep_form != s_rep_form:
            if " " in i_rep_form and not " " in s_rep_form and i_rep_form.count(" ") == 1:
                poss = i_segs[i_rep_form]['pos']
                check = sorted(poss.values())
                if check[-1] / sum(check) < 0.8:
                    co_dict[item] = big_dict[item]
                    co_count += 1
                else:
                    correct_form = max(poss, key=poss.get)
                    correct_form = ict_pku(correct_form)
                    
                    correct_form_str = ""
                    correct_form_temp = correct_form.split(" ")
                    for cft in correct_form_temp:
                        correct_form_str += cft[:cft.index("/")] + " "
                    correct_form_str = correct_form_str[:-1]
                    
                    if '/u' in correct_form:
                        g = cciu.write("// " + item + "\n")
                        cciu_count += 1
                        all_segs = set()
                        for k in i_cons:
                            all_segs.add(k)
                        for k in s_cons:
                            all_segs.add(k)
                        for ele in all_segs:
                            if correct_form_str != ele:
                                any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
                                any_str_space = any_str_plus.replace("+", " ")
                                config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
                                g = cciu.write(config_str + "\n")
                            else:
                                wl_seg = ele.split(" ")
                                corr_p = correct_form.split(" ")
                                corr_p1 = corr_p[0][corr_p[0].index("/")+1:]
                                corr_p2 = corr_p[1][corr_p[1].index("/")+1:]
                                g = cciu.write("KEY" + wl_seg[0] + "/ANY+" + wl_seg[1] + "/ANY->ANY>>" + corr_p1 + '\n')
                                g = cciu.write(wl_seg[0] + "/ANY+KEY" + wl_seg[1] + "/ANY->ANY>>" + corr_p2 + '\n')  
                        g = cciu.write('\n')
                    elif '/p' in correct_form:
                        g = ccip.write("// " + item + "\n")
                        ccip_count += 1
                        all_segs = set()
                        for k in i_cons:
                            all_segs.add(k)
                        for k in s_cons:
                            all_segs.add(k)
                        for ele in all_segs:
                            if correct_form_str != ele:
                                any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
                                any_str_space = any_str_plus.replace("+", " ")
                                config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
                                g = ccip.write(config_str + "\n")
                            else:
                                wl_seg = ele.split(" ")
                                corr_p = correct_form.split(" ")
                                corr_p1 = corr_p[0][corr_p[0].index("/")+1:]
                                corr_p2 = corr_p[1][corr_p[1].index("/")+1:]
                                g = ccip.write("KEY" + wl_seg[0] + "/ANY+" + wl_seg[1] + "/ANY->ANY>>" + corr_p1 + '\n')
                                g = ccip.write(wl_seg[0] + "/ANY+KEY" + wl_seg[1] + "/ANY->ANY>>" + corr_p2 + '\n')  
                        g = ccip.write('\n')
                    elif '/f' in correct_form:
                        g = ccif.write("// " + item + "\n")
                        ccif_count += 1
                        all_segs = set()
                        for k in i_cons:
                            all_segs.add(k)
                        for k in s_cons:
                            all_segs.add(k)
                        for ele in all_segs:
                            if correct_form_str != ele:
                                any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
                                any_str_space = any_str_plus.replace("+", " ")
                                config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
                                g = ccif.write(config_str + "\n")
                            else:
                                wl_seg = ele.split(" ")
                                corr_p = correct_form.split(" ")
                                corr_p1 = corr_p[0][corr_p[0].index("/")+1:]
                                corr_p2 = corr_p[1][corr_p[1].index("/")+1:]
                                g = ccif.write("KEY" + wl_seg[0] + "/ANY+" + wl_seg[1] + "/ANY->ANY>>" + corr_p1 + '\n')
                                g = ccif.write(wl_seg[0] + "/ANY+KEY" + wl_seg[1] + "/ANY->ANY>>" + corr_p2 + '\n')  
                        g = ccif.write('\n')
                    elif '/y' in correct_form:
                        g = cciy.write("// " + item + "\n")
                        cciy_count += 1
                        all_segs = set()
                        for k in i_cons:
                            all_segs.add(k)
                        for k in s_cons:
                            all_segs.add(k)
                        for ele in all_segs:
                            if correct_form_str != ele:
                                any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
                                any_str_space = any_str_plus.replace("+", " ")
                                config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
                                g = cciy.write(config_str + "\n")
                            else:
                                wl_seg = ele.split(" ")
                                corr_p = correct_form.split(" ")
                                corr_p1 = corr_p[0][corr_p[0].index("/")+1:]
                                corr_p2 = corr_p[1][corr_p[1].index("/")+1:]
                                g = cciy.write("KEY" + wl_seg[0] + "/ANY+" + wl_seg[1] + "/ANY->ANY>>" + corr_p1 + '\n')
                                g = cciy.write(wl_seg[0] + "/ANY+KEY" + wl_seg[1] + "/ANY->ANY>>" + corr_p2 + '\n')  
                        g = cciy.write('\n')
                    elif '/m' in correct_form:
                        g = ccim.write("// " + item + "\n")
                        ccim_count += 1
                        all_segs = set()
                        for k in i_cons:
                            all_segs.add(k)
                        for k in s_cons:
                            all_segs.add(k)
                        for ele in all_segs:
                            if correct_form_str != ele:
                                any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
                                any_str_space = any_str_plus.replace("+", " ")
                                config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
                                g = ccim.write(config_str + "\n")
                            else:
                                wl_seg = ele.split(" ")
                                corr_p = correct_form.split(" ")
                                corr_p1 = corr_p[0][corr_p[0].index("/")+1:]
                                corr_p2 = corr_p[1][corr_p[1].index("/")+1:]
                                g = ccim.write("KEY" + wl_seg[0] + "/ANY+" + wl_seg[1] + "/ANY->ANY>>" + corr_p1 + '\n')
                                g = ccim.write(wl_seg[0] + "/ANY+KEY" + wl_seg[1] + "/ANY->ANY>>" + corr_p2 + '\n')  
                        g = ccim.write('\n')
                    else:
                        co_dict[item] = big_dict[item]
                        co_count += 1                
                
                           
            elif " " in s_rep_form and not " " in i_rep_form and s_rep_form.count(" ") == 1:
                poss = s_segs[s_rep_form]['pos']
                check = sorted(poss.values())
                if check[-1] / sum(check) < 0.8:
                    co_dict[item] = big_dict[item]
                    co_count += 1
                else:
                    correct_form = max(poss, key=poss.get)
                    correct_form = ict_pku(correct_form)
                    
                    correct_form_str = ""
                    correct_form_temp = correct_form.split(" ")
                    for cft in correct_form_temp:
                        correct_form_str += cft[:cft.index("/")] + " "
                    correct_form_str = correct_form_str[:-1]
                    
                    if '/u' in correct_form:
                        g = ccsu.write("// " + item + "\n")
                        ccsu_count += 1
                        all_segs = set()
                        for k in i_cons:
                            all_segs.add(k)
                        for k in s_cons:
                            all_segs.add(k)
                        for ele in all_segs:
                            if correct_form_str != ele:
                                any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
                                any_str_space = any_str_plus.replace("+", " ")
                                config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
                                g = ccsu.write(config_str + "\n")
                            else:
                                wl_seg = ele.split(" ")
                                corr_p = correct_form.split(" ")
                                corr_p1 = corr_p[0][corr_p[0].index("/")+1:]
                                corr_p2 = corr_p[1][corr_p[1].index("/")+1:]
                                g = ccsu.write("KEY" + wl_seg[0] + "/ANY+" + wl_seg[1] + "/ANY->ANY>>" + corr_p1 + '\n')
                                g = ccsu.write(wl_seg[0] + "/ANY+KEY" + wl_seg[1] + "/ANY->ANY>>" + corr_p2 + '\n')  
                        g = ccsu.write('\n')
                    elif '/p' in correct_form:
                        g = ccsp.write("// " + item + "\n")
                        ccsp_count += 1
                        all_segs = set()
                        for k in i_cons:
                            all_segs.add(k)
                        for k in s_cons:
                            all_segs.add(k)
                        for ele in all_segs:
                            if correct_form_str != ele:
                                any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
                                any_str_space = any_str_plus.replace("+", " ")
                                config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
                                g = ccsp.write(config_str + "\n")
                            else:
                                wl_seg = ele.split(" ")
                                corr_p = correct_form.split(" ")
                                corr_p1 = corr_p[0][corr_p[0].index("/")+1:]
                                corr_p2 = corr_p[1][corr_p[1].index("/")+1:]
                                g = ccsp.write("KEY" + wl_seg[0] + "/ANY+" + wl_seg[1] + "/ANY->ANY>>" + corr_p1 + '\n')
                                g = ccsp.write(wl_seg[0] + "/ANY+KEY" + wl_seg[1] + "/ANY->ANY>>" + corr_p2 + '\n')  
                        g = ccsp.write('\n')
                    elif '/f' in correct_form:
                        g = ccsf.write("// " + item + "\n")
                        ccsf_count += 1
                        all_segs = set()
                        for k in i_cons:
                            all_segs.add(k)
                        for k in s_cons:
                            all_segs.add(k)
                        for ele in all_segs:
                            if correct_form_str != ele:
                                any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
                                any_str_space = any_str_plus.replace("+", " ")
                                config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
                                g = ccsf.write(config_str + "\n")
                            else:
                                wl_seg = ele.split(" ")
                                corr_p = correct_form.split(" ")
                                corr_p1 = corr_p[0][corr_p[0].index("/")+1:]
                                corr_p2 = corr_p[1][corr_p[1].index("/")+1:]
                                g = ccsf.write("KEY" + wl_seg[0] + "/ANY+" + wl_seg[1] + "/ANY->ANY>>" + corr_p1 + '\n')
                                g = ccsf.write(wl_seg[0] + "/ANY+KEY" + wl_seg[1] + "/ANY->ANY>>" + corr_p2 + '\n')  
                        g = ccsf.write('\n')
                    elif '/y' in correct_form:
                        g = ccsy.write("// " + item + "\n")
                        ccsy_count += 1
                        all_segs = set()
                        for k in i_cons:
                            all_segs.add(k)
                        for k in s_cons:
                            all_segs.add(k)
                        for ele in all_segs:
                            if correct_form_str != ele:
                                any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
                                any_str_space = any_str_plus.replace("+", " ")
                                config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
                                g = ccsy.write(config_str + "\n")
                            else:
                                wl_seg = ele.split(" ")
                                corr_p = correct_form.split(" ")
                                corr_p1 = corr_p[0][corr_p[0].index("/")+1:]
                                corr_p2 = corr_p[1][corr_p[1].index("/")+1:]
                                g = ccsy.write("KEY" + wl_seg[0] + "/ANY+" + wl_seg[1] + "/ANY->ANY>>" + corr_p1 + '\n')
                                g = ccsy.write(wl_seg[0] + "/ANY+KEY" + wl_seg[1] + "/ANY->ANY>>" + corr_p2 + '\n')  
                        g = ccsy.write('\n')
                    elif '/m' in correct_form:
                        g = ccsm.write("// " + item + "\n")
                        ccsm_count += 1
                        all_segs = set()
                        for k in i_cons:
                            all_segs.add(k)
                        for k in s_cons:
                            all_segs.add(k)
                        for ele in all_segs:
                            if correct_form_str != ele:
                                any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
                                any_str_space = any_str_plus.replace("+", " ")
                                config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
                                g = ccsm.write(config_str + "\n")
                            else:
                                wl_seg = ele.split(" ")
                                corr_p = correct_form.split(" ")
                                corr_p1 = corr_p[0][corr_p[0].index("/")+1:]
                                corr_p2 = corr_p[1][corr_p[1].index("/")+1:]
                                g = ccsm.write("KEY" + wl_seg[0] + "/ANY+" + wl_seg[1] + "/ANY->ANY>>" + corr_p1 + '\n')
                                g = ccsm.write(wl_seg[0] + "/ANY+KEY" + wl_seg[1] + "/ANY->ANY>>" + corr_p2 + '\n')  
                        g = ccsm.write('\n')
                    else:
                        co_dict[item] = big_dict[item]
                        co_count += 1
            else:
                co_dict[item] = big_dict[item]
                co_count += 1
#   模棱两可            
    else:
        pass
#     elif i_con + s_con < 0.65
#         i_dic = i_segs
#         s_dic = s_segs

#         need_config = False
#         for segs in i_dic:
#             max_p = ""
#             p_num = 0
#             for pos in i_dic[segs]['pos']:
#                 if i_dic[segs]['pos'][pos] > p_num:
#                     max_p = pos

#             chars = max_p.split(" ")
#             if len(chars) == 2 and '/r' in chars[0] and 'n' in chars[1]:
#                 need_config = True
                
                
                
                
                
                
#         for segs in s_dic:
#             max_p = ""
#             p_num = 0
#             for pos in s_dic[segs]['pos']:
#                 if s_dic[segs]['pos'][pos] > p_num:
#                     max_p = pos

#             chars = max_p.split(" ")
#             if len(chars) == 2 and '/r' in chars[0] and 'n' in chars[1]:
#                 need_config = True
#         if need_config:
#             g = cu.write(k + "\n")
#             g = cu.write(str(big_dict[k]) + '\n')
#             correct_exp = k[0] + '/r' + ' ' + k[1] + '/n'
#             pos_set = set()
#             for segs in i_dic:
#                 for pos in i_dic[segs]['pos']:
#                     pos_set.add(pos)
#             for item in pos_set:
#                 g = cu.write('KEY' + item.replace(" ", "+") + "->[" + item + "]MERGE[" + correct_exp + "]\n")
#             g = cu.write("\n")
    
    
    
    
    
    
    
    
    
    
    
    
#         i_rep_form = max(i_cons, key=i_cons.get)
#         s_rep_form = max(s_cons, key=s_cons.get)
#         if " " in i_rep_form or not " " in s_rep_form and i_rep_form.count(" ") == 1:
#                 poss = i_segs[i_rep_form]['pos']
#                 check = sorted(poss.values())
#                 if check[-1] / sum(check) < 0.8:
#                     co_dict[item] = big_dict[item]
#                     co_count += 1
#                 else:
#                     correct_form = max(poss, key=poss.get)
#                     correct_form = ict_pku(correct_form)
                    
#                     correct_form_str = ""
#                     correct_form_temp = correct_form.split(" ")
#                     for cft in correct_form_temp:
#                         correct_form_str += cft[:cft.index("/")] + " "
#                     correct_form_str = correct_form_str[:-1]
        
        
        
#         if '/r' in correct_form.split(" ")[0] and 'n' in correct_form.split(" ")[0]:
#             g = rn.write("// " + item + "\n")
#             rn_count += 1
#             all_segs = set()
#             for k in i_cons:
#                 all_segs.add(k)
#             for k in s_cons:
#                 all_segs.add(k)
#             for ele in all_segs:
#                 if correct_form_str != ele:
#                     any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
#                     any_str_space = any_str_plus.replace("+", " ")
#                     config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
#                     g = rn.write(config_str + "\n")
#         elif '/d' in correct_form.split(" ")[0] and '/v' in correct_form.split(" ")[0]:
#             g = ad_v.write("// " + item + "\n")
#             ad_v_count += 1
#             all_segs = set()
#             for k in i_cons:
#                 all_segs.add(k)
#             for k in s_cons:
#                 all_segs.add(k)
#             for ele in all_segs:
#                 if correct_form_str != ele:
#                     any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
#                     any_str_space = any_str_plus.replace("+", " ")
#                     config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
#                     g = ad_v.write(config_str + "\n")
#         elif '/d' in correct_form.split(" ")[0] and '/a' in correct_form.split(" ")[0]:
#             g = ad_a.write("// " + item + "\n")
#             ad_a_count += 1
#             all_segs = set()
#             for k in i_cons:
#                 all_segs.add(k)
#             for k in s_cons:
#                 all_segs.add(k)
#             for ele in all_segs:
#                 if correct_form_str != ele:
#                     any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
#                     any_str_space = any_str_plus.replace("+", " ")
#                     config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
#                     g = ad_a.write(config_str + "\n")
#         elif '/d' in correct_form.split(" ")[0] and '/p' in correct_form.split(" ")[0]:
#             g = ad_p.write("// " + item + "\n")
#             ad_p_count += 1
#             all_segs = set()
#             for k in i_cons:
#                 all_segs.add(k)
#             for k in s_cons:
#                 all_segs.add(k)
#             for ele in all_segs:
#                 if correct_form_str != ele:
#                     any_str_plus = ''.join(p+"/ANY+" for p in ele.split(" "))[:-1]
#                     any_str_space = any_str_plus.replace("+", " ")
#                     config_str = "KEY" + any_str_plus + "->[" + any_str_space + "]MERGE[" + correct_form + "]"
#                     g = ad_p.write(config_str + "\n")
#         else:
#             en_dict[item] = big_dict[item]
    
    
#     else：
#         en_dict[item] = big_dict[item]
    
    
    
    
    
g = co.write(str(co_dict))  
g = en.write(str(en_dict))
    
nd.close()
wd.close()
co.close()
cciu.close()
ccip.close()
ccif.close()
cciy.close()
ccim.close()
ccsu.close()
ccsp.close()
ccsf.close()
ccsy.close()
ccsm.close()
rn.close()
ad_v.close()
ad_a.close()
ad_p.close()
en.close()

print("nd_count: " + str(nd_count))
print("wd_count: " + str(wd_count))
print("co_count: " + str(co_count))

print("cciu_count: " + str(cciu_count))
print("ccip_count: " + str(ccip_count))
print("ccif_count: " + str(ccif_count))
print("cciy_count: " + str(cciy_count))
print("ccim_count: " + str(ccim_count))
print("ccsu_count: " + str(ccsu_count))
print("ccsp_count: " + str(ccsp_count))
print("ccsf_count: " + str(ccsf_count))
print("ccsy_count: " + str(ccsy_count))
print("ccsm_count: " + str(ccsm_count))

print("rn: " + str(rn_count))
print("ad_v: " + str(ad_v_count))
print("ad_a: " + str(ad_a_count))
print("ad_p: " + str(ad_p_count))
print("en: " + str(en_count))
    

* nd_count: 683
* wd_count: 3559
* co_count: 10119
* cciu_count: 170
* ccip_count: 223
* ccif_count: 273
* cciy_count: 17
* ccim_count: 134
* ccsu_count: 12
* ccsp_count: 2
* ccsf_count: 8
* ccsy_count: 0
* ccsm_count: 80
* rn: 0
* ad_v: 0
* ad_a: 0
* ad_p: 0
* en: 0

## need_discuss_confident 已经差不多解决
## need_discuss_conflict 是有歧义的分词形况中还没有解决的

# 中华百家姓

In [ ]:
ln = open("config_lastname.txt", 'w', encoding="utf-8-sig")
with open("中华百家姓.txt", encoding="utf-8-sig") as n:
    for line in n:
        line = line[:-1]
        if len(line) == 1:
            g = ln.write("KEY" + line + "某/ANY->["+line+"某/ANY]MERGE["+line + "/nr 某/r]\n")
            g = ln.write("KEY小" + line + "/ANY->[小"+line+"/ANY]MERGE[小/a "+line + "/nr]\n")
            g = ln.write("KEY老" + line + "/ANY->[老"+line+"/ANY]MERGE[老/a "+line + "/nr]\n")
            g = ln.write("\n")
ln.close()

### 中华百家姓的config中会有些本身有意义的词 如 "老马, 老师, 小云"等等，人工将其剔除